<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.62
    Uninstalling yfinance-0.2.62:
      Successfully uninstalled yfinance-0.2.62
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-09 11:05:57
-------------------
qualified stocks: 88
with latest results: 29
still star stocks: 16
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.11 L
Current:  1.35 C
-------------------
Today PnL: 1.43 L (1.07%)
Current PnL: -14.89 L (-10.38%)
CY Booked + Current PnL: -8.81 L (-6.14%)
-------------------
Total profit:  3.96 L
Total loss:  -18.86 L
-------------------
Total Booked + Current PnL: 18.17 L (15.59%)
Total Booked PnL: 33.06 L (28.37%)
Curr Year Booked PnL: 6.08 L (4.51%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.05 C
Est FTT PnL: 69.83 L (51.79%)
Deployed:  1.17 C
Current:  1.35 C
CAGR/XIRR %: 10.34%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LICI,785.05,983.0,-2.27,H-LC,85.23,239531.0,44839.0,4264.0,0.63,23.03,1.78,25.21,31.0,10.52,1.91,32.45,XY25,NTT,INSURANCE
80,UNIONBANK,123.87,163.0,1.23,M-LC,93.18,178338.0,37498.0,6991.0,2.33,26.62,3.92,31.59,81.0,5.36,1.42,54.91,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,3.96,M-LC,57.95,104384.0,26022.0,8006.0,-0.10,33.21,7.67,43.43,71.0,3.25,0.83,39.61,XY24,NTT,MISC
57,RELIANCE,1291.83,1526.0,6.50,X-LC,35.23,157974.0,17165.0,8357.0,0.40,12.19,5.29,18.13,20.0,2.05,1.26,24.74,XY25,NTT,REFINERIES
73,TATAELXSI,7332.28,7332.0,-12.96,X-MC,64.77,91476.0,-11176.0,11169.0,0.94,-10.89,12.21,-0.00,51.0,-1.00,0.73,37.50,OX40N,NTT,IT


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-57.75,L-SC,56.82,63494.0,-50055.0,90104.0,9.98,-44.08,141.91,35.27,246.0,-0.56,0.51,55.40,XR,NTT,HOTELS
11,BANDHANBNK,214.25,400.00,-27.77,H-SC,80.68,215238.0,-33292.0,248772.0,6.91,-13.40,115.58,86.70,118.0,-0.13,1.71,42.39,XY24,NTT,BANKS
86,WHIRLPOOL,1167.49,2270.00,-31.63,M-SC,52.27,199494.0,27873.0,134200.0,5.33,16.24,67.27,94.43,183.0,0.21,1.59,47.24,XR,NTT,DURABLES
55,RBLBANK,230.61,388.00,49.48,M-SC,72.73,232847.0,-1453.0,161363.0,5.31,-0.62,69.30,68.25,229.0,-0.01,1.85,53.34,XY24,NTT,BANKS
29,HAPPSTMNDS,919.80,1491.84,-22.75,M-SC,44.32,65042.0,-31537.0,91599.0,5.19,-32.65,140.83,62.19,195.0,-0.34,0.52,14.05,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BERGEPAINT,558.54,680.00,-15.50,H-MC,42.05,181324.0,4267.0,34234.0,-1.63,2.41,18.88,21.75,106.0,0.12,1.44,29.71,XY24,NTT,PAINTS
17,CAMPUS,294.86,393.00,NaN,NaN,86.36,168481.0,706.0,55144.0,-0.92,0.42,32.73,33.28,188.0,0.01,1.34,30.44,XY24,NTT,FOOTWEAR
37,INDIAMART,2327.09,4871.06,-51.09,H-SC,47.73,127529.0,4193.0,130641.0,-0.90,3.40,102.44,109.32,111.0,0.03,1.01,25.99,AR,ATH,MISC
6,APOLLOHOSP,6904.43,8285.00,-16.11,H-LC,22.73,103620.0,54.0,20651.0,-0.50,0.05,19.93,20.00,33.0,0.00,0.82,14.39,X40N,BTT,HEALTHCARE
67,SIS,477.00,477.00,2212.56,M-SC,61.36,62611.0,-18479.0,18477.0,-0.45,-22.79,29.51,0.00,211.0,-1.00,0.50,26.89,OX40N,NTT,MISC


In [12]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,52.77,53.0,49.36,M-SC,78.41,129600.0,-12879.0,13504.0,0.36,-9.04,10.42,0.44,227.0,-0.95,1.03,52.96,OX40N,NTT,BANKS
71,SYMPHONY,1306.42,1306.0,-28.04,M-SC,1.14,153270.0,-17871.0,17810.0,1.69,-10.44,11.62,-0.03,178.0,-1.00,1.22,10.16,OX40N,NTT,DURABLES
73,TATAELXSI,7332.28,7332.0,-12.96,X-MC,64.77,91476.0,-11176.0,11169.0,0.94,-10.89,12.21,-0.00,51.0,-1.00,0.73,37.50,OX40N,NTT,IT
45,KANSAINER,299.63,340.0,-67.67,H-SC,6.82,225000.0,-44667.0,81000.0,0.34,-16.56,36.00,13.47,134.0,-0.55,1.79,12.62,XY24,NTT,PAINTS
78,TTKPRESTIG,769.29,770.0,-10.88,X-SC,17.05,81534.0,-19243.0,19332.0,0.17,-19.09,23.71,0.09,64.0,-1.00,0.65,6.10,OX40N,NTT,DURABLES


In [13]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,3794.03,4998.00,-18.09,X-LC,13.64,266776.0,-29158.0,123064.0,1.06,-9.85,46.13,31.73,1.0,-0.24,2.12,6.78,X40,BTT,IT
40,INFY,1461.46,2275.00,-12.06,X-LC,38.64,282176.0,20575.0,125060.0,0.81,7.86,44.32,55.67,2.0,0.16,2.24,14.45,X40,BTT,IT
49,LTIM,5564.16,7197.33,-1.63,X-LC,69.32,189432.0,-10878.0,69673.0,1.11,-5.43,36.78,29.35,3.0,-0.16,1.51,30.87,X200,ATH,IT
7,ASIANPAINT,2961.56,4250.00,-23.00,X-LC,7.95,153000.0,-48386.0,136002.0,0.21,-24.03,88.89,43.51,5.0,-0.36,1.22,5.53,X40,BTT,PAINTS
33,HINDUNILVR,2413.81,2723.00,-14.53,X-LC,21.59,268397.0,-1950.0,36583.0,0.28,-0.72,13.63,12.81,6.0,-0.05,2.14,11.08,XY25,NTT,FMCG


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UNIONBANK,123.87,163.0,1.23,M-LC,93.18,178338.0,37498.0,6991.0,2.33,26.62,3.92,31.59,81.0,5.36,1.42,54.91,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,3.96,M-LC,57.95,104384.0,26022.0,8006.0,-0.10,33.21,7.67,43.43,71.0,3.25,0.83,39.61,XY24,NTT,MISC
48,LICI,785.05,983.0,-2.27,H-LC,85.23,239531.0,44839.0,4264.0,0.63,23.03,1.78,25.21,31.0,10.52,1.91,32.45,XY25,NTT,INSURANCE


In [15]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,130.64,228.00,104.28,M-MC,89.77,232177.0,7215.0,160434.0,-0.16,3.21,69.10,74.53,176.0,0.04,1.85,34.80,XY24,BTT,STEEL
18,CAMS,3643.00,5250.99,6.89,H-SC,88.07,120061.0,18057.0,26966.0,0.61,17.70,22.46,44.14,115.0,0.67,0.96,36.82,X40N,ATH,MISC
4,ADANIPORTS,1103.69,1583.00,3.96,M-LC,57.95,104384.0,26022.0,8006.0,-0.10,33.21,7.67,43.43,71.0,3.25,0.83,39.61,XY24,NTT,MISC
80,UNIONBANK,123.87,163.00,1.23,M-LC,93.18,178338.0,37498.0,6991.0,2.33,26.62,3.92,31.59,81.0,5.36,1.42,54.91,XY24,NTT,BANKS
87,WIPRO,243.46,420.00,-9.71,M-LC,23.86,154771.0,3826.0,105631.0,0.42,2.53,68.25,72.51,68.0,0.04,1.23,7.14,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,113.49,190.00,-24.57,M-MC,75.00,147132.0,17980.0,69093.0,3.95,13.92,46.96,67.42,158.0,0.26,1.17,43.10,XR,NTT,BANKS
86,WHIRLPOOL,1167.49,2270.00,-31.63,M-SC,52.27,199494.0,27873.0,134200.0,5.33,16.24,67.27,94.43,183.0,0.21,1.59,47.24,XR,NTT,DURABLES
87,WIPRO,243.46,420.00,-9.71,M-LC,23.86,154771.0,3826.0,105631.0,0.42,2.53,68.25,72.51,68.0,0.04,1.23,7.14,XR,NTT,IT
37,INDIAMART,2327.09,4871.06,-51.09,H-SC,47.73,127529.0,4193.0,130641.0,-0.90,3.40,102.44,109.32,111.0,0.03,1.01,25.99,AR,ATH,MISC
1,AARTIIND,487.04,919.00,40.47,M-SC,43.18,80719.0,-617.0,72752.0,1.36,-0.76,90.13,88.69,213.0,-0.01,0.64,36.12,XR,NTT,CHEMICALS


In [17]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-57.75,L-SC,56.82,63494.0,-50055.0,90104.0,9.98,-44.08,141.91,35.27,246.0,-0.56,0.51,55.40,XR,NTT,HOTELS
5,ALKYLAMINE,2347.98,4567.52,-7.04,X-SC,88.07,86507.0,-14456.0,109898.0,1.52,-14.32,127.04,94.53,58.0,-0.13,0.69,28.25,SR,ATH,CHEMICALS
50,MASFIN,326.60,399.50,-19.60,H-SC,73.86,91140.0,-6840.0,28709.0,3.07,-6.98,31.50,22.32,127.0,-0.24,0.73,32.18,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.00,3.96,M-LC,57.95,104384.0,26022.0,8006.0,-0.10,33.21,7.67,43.43,71.0,3.25,0.83,39.61,XY24,NTT,MISC
82,VALIANTORG,512.64,838.00,-363.88,H-SC,100.00,112190.0,-21096.0,105694.0,-0.05,-15.83,94.21,63.47,135.0,-0.20,0.89,88.06,XR,NTT,CHEMICALS


In [18]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VALIANTORG,512.64,838.00,-363.88,H-SC,100.00,112190.0,-21096.0,105694.0,-0.05,-15.83,94.21,63.47,135.0,-0.20,0.89,88.06,XR,NTT,CHEMICALS
8,ASIANTILES,75.41,137.00,6566.67,M-SC,98.86,73147.0,-20663.0,97278.0,-0.36,-22.03,132.99,81.67,222.0,-0.21,0.58,41.65,XR,NTT,CERAMICS
76,TITAGARH,1097.23,1548.00,-5.82,H-SC,97.73,165515.0,-26500.0,105383.0,0.52,-13.80,63.67,41.08,131.0,-0.25,1.32,40.09,XY24,NTT,ENGINEERING
72,TANLA,917.30,1963.11,-33.39,H-SC,96.59,163543.0,-75872.0,348837.0,0.60,-31.69,213.30,114.01,117.0,-0.22,1.30,49.99,AR,ATH,IT
69,STARHEALTH,551.56,761.00,4.17,M-SC,95.45,170483.0,-29733.0,105768.0,-0.07,-14.85,62.04,37.97,202.0,-0.28,1.36,36.88,XY24,NTT,INSURANCE


In [19]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.43
1,25,43.85
2,50,73.43


In [20]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     33.64
XY25     15.41
XR       11.60
X40      10.62
X40N      8.91
OX40N     8.74
AR        8.34
X200      1.51
SR        1.24
Name: CurrAlloc%, dtype: float64

In [21]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.62
X-LC    19.02
M-SC    17.48
H-MC     8.20
M-LC     8.12
H-LC     7.48
M-MC     5.82
X-MC     5.78
X-SC     2.91
L-SC     2.50
L-LC     1.19
Name: CurrAlloc%, dtype: float64

In [22]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.11,-1.05,32.69
IT,10.78,-15.15,75.97
BANKS,10.60,-5.06,51.94
MISC,6.80,-26.45,75.89
FINANCE,6.38,-19.13,65.59
INSURANCE,5.59,5.06,23.31
PAINTS,5.52,-18.67,42.11
ELECTRICAL,5.43,-2.98,47.65
HEALTHCARE,4.55,-3.71,34.66


In [23]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2744426.0,26
XR,1179983.0,14
AR,1021502.0,9
X40,593559.0,7
XY25,443128.0,10
X40N,409295.0,8
OX40N,331735.0,11
SR,189513.0,2
X200,69673.0,1


In [24]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1899479.0,18
M-SC,1677059.0,20
X-LC,821529.0,11
M-MC,536493.0,5
H-MC,466596.0,7
M-LC,285211.0,6
X-MC,283970.0,5
L-SC,283903.0,4
X-SC,267217.0,4


In [25]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,882197.0,6
M-SC,XY24,729370.0,6
H-SC,AR,538744.0,3
X-LC,X40,467154.0,4
M-SC,XR,421038.0,5
M-MC,XY24,381929.0,3
H-SC,XR,206552.0,3
H-MC,XY24,172215.0,2
M-LC,XY24,168379.0,4


In [26]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 45.0 seconds
